# Installing dependencies


In [1]:
!pip install geopandas
!pip install plotly_express
!pip install tqdm
!pip install geopy

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 277kB/s 
     |████████████████████████████████| 10.9MB 45.6MB/s 


# Importing libraries and packages

In [2]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
from google.colab import drive 
drive.mount('/content/gdrive')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Function that converts latitude and longitude into respective zip code using GeoPy and OpenStreetMap's Nominatim API service


In [0]:
def reversegeo(lat, lon):
  locator = Nominatim(user_agent="myGeocoder")
  coordinates = str(lat)+","+str(lon)
  location = locator.reverse(coordinates)
  if "postcode" in location.raw['address']:
    return location.raw['address']['postcode']
  else: return "NA"

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls "/content/drive/My Drive/Mobility data"

202003-citibike-tripdata.csv  march_citiends.csv  march_citistarts.csv


In [8]:
import pandas as pd

subway = pd.io.gbq.read_gbq('''
 SELECT s1.*,s2.station_lat, s2.station_lon FROM `bigquery-public-data.new_york_subway.subway_ridership_2013_present` s1
inner join `bigquery-public-data.new_york_subway.stations` s2
on s1.station_name=s2.station_name
WHERE s2.station_lat IS NOT NULL AND s2.station_lon IS NOT NULL
''', project_id='mit-covid-19-datathon', dialect='standard')



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=kyVmqRabKXqOeYap1Csnw2tNCJzZMU&prompt=consent&access_type=offline
Enter the authorization code: 4/zwFL_u6R022wWLKvD8h7bg_yvhT2LiGMpegkugz1La9-gRGnN5i-PlQ


## Applying the reversegeocoding function to obtain zipcode for subway station latitude and longitude


In [0]:
subway['zipcode']=subway.apply(lambda x: reversegeo(x['station_lat'], x['station_lon']), axis=1)

## Truncating zipcodes longer than 5 digits to 5 digits

In [0]:
 subway['zipcode']=subway['zipcode'].apply(lambda x: x[:5])


## Removing zip codes which are NA and dropping the latitude,longitude, and duplicates

In [0]:
 subway=subway[subway['zipcode']!='NA']
 subway=subway.drop(['station_lat','station_lon'],axis=1)
 subway=subway.drop_duplicates()

## Aggregating the number of riders for each year and averaging it over a 5 year period

In [0]:
newdf=subway.groupby(['zipcode'])['ridership_2013'].sum().reset_index()

In [0]:
newdf['station_count']=subway.groupby(['zipcode'])['station_name'].nunique().reset_index()['station_name']
newdf['ridership_2014']=subway.groupby(['zipcode'])['ridership_2014'].sum().reset_index()['ridership_2014']
newdf['ridership_2015']=subway.groupby(['zipcode'])['ridership_2015'].sum().reset_index()['ridership_2015']
newdf['ridership_2016']=subway.groupby(['zipcode'])['ridership_2016'].sum().reset_index()['ridership_2016']
newdf['ridership_2017']=subway.groupby(['zipcode'])['ridership_2017'].sum().reset_index()['ridership_2017']
newdf['ridership_2018']=subway.groupby(['zipcode'])['ridership_2018'].sum().reset_index()['ridership_2018']
newdf['average_riders']=(newdf['ridership_2013']+newdf['ridership_2014']+newdf['ridership_2015']+newdf['ridership_2016']+newdf['ridership_2017']+newdf['ridership_2018'])/5

In [0]:
subwaydf=newdf[['zipcode','station_count','average_riders']]

In [0]:
subwaydf.to_csv('subway_updated.csv')